In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier

In [4]:
class PSSMDataset(torch.utils.data.Dataset):
    def __init__(self,pssm_in,label_in):
        self.pssm_lst = sorted(os.listdir(pssm_in))
        self.pssm_files = [pssm_in+f for f in self.pssm_lst]
        self.label_files = [label_in+f.split('.')[0]+'.label.txt' for f in self.pssm_lst]
    def __len__(self):
        return len(self.label_files)
    def __getitem__(self, idx):
        self.pssm_tensor = torch.tensor(pd.read_table(self.pssm_files[idx],index_col=0).astype(float).values)
        self.label_tensor = torch.tensor(list(map(int,open(self.label_files[idx],'r').readlines()[0].split(','))))
        return self.pssm_tensor, self.label_tensor

def my_collate(batch):
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    dim_data = [d.size()[0] for d in data]
    max_seqlen = max(dim_data)
    data_transformed = [F.pad(d.T,(0,int(max_seqlen - d.size()[0])),'constant',0.0).T if d.size()[0]!=max_seqlen else d for d in data]
    target_transformed = [F.pad(d,(0,int(max_seqlen - d.size()[0])),'constant',0.0) if d.size()[0]!=max_seqlen else d for d in target]
    return [data_transformed, target_transformed]

pssminpath = '/Users/suhancho/data/Uniprot_metalbinding_challenge/processed_pssm/'
labelinpath = '/Users/suhancho/data/Uniprot_metalbinding_challenge/processed_label/'
train_ds = PSSMDataset(pssminpath,labelinpath)
train_dl = DataLoader(train_ds,batch_size = 4,shuffle = True,collate_fn=my_collate)



In [5]:
clf = TabNetClassifier(cat_emb_dim=10,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-2),
                       scheduler_params={"step_size":50,
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='sparsemax' # "sparsemax", entmax
                      )

/Users/suhancho/miniforge3/envs/pytorch/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [ ]:
max_epochs = 15

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False,
)